# Lecture 4: Structure your model

###  Phase 1: Assemble graph
1. Define placeholders for input and output
2. Define the weights
3. Define the inference model
4. Define loss function
5. Define optimizer

# Word2vec example: skipgram

## with embeddings visualization

In [36]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
from util import newlogname
from process_data import process_data
import sys
import os
from util import newlogname

class Config(object):
    """Holds model hyperparams and data information.
    The config class is used to store various hyperparameters and dataset
    information parameters. Model objects are passed a Config() object at
    instantiation.
    """
    def __init__(self,
                 vocab_size=50000,
                 batch_size=128,
                 embed_size=128,
                 skip_window=1,
                 num_sampled=64,
                 lr=0.00217346380124,
                 num_train_steps=100000,
                 skip_step=2000,
                 ):
        self.vocab_size = vocab_size
        self.batch_size = batch_size
        self.embed_size = embed_size
        self.skip_window = skip_window
        self.num_sampled = num_sampled
        self.lr = lr
        self.num_train_steps = num_train_steps
        self.skip_step = skip_step
        self.log_path = newlogname()


class SkipGramModel(object):
    """ Build the graph for word2vec model """
    def __init__(self, config):
        self.config = config
        self.build_graph()

    def _create_placeholders(self):
        with tf.name_scope("data"):
            self.center_words = tf.placeholder(tf.int32,
                                               shape=[self.config.batch_size],
                                               name='center_words')
            self.target_words = tf.placeholder(tf.int32,
                                               shape=[self.config.batch_size,
                                                      1],
                                               name='target_words')

    def _create_embedding(self):
        with tf.name_scope("embed"):
            emshape = (self.config.vocab_size, self.config.embed_size)
            eminit = tf.random_uniform(shape=emshape, minval=-1, maxval=1)
            self.embed_matrix = tf.get_variable("embed_matrix",
                                                dtype=tf.float32,
                                                initializer=eminit)

    def _create_loss(self):
        with tf.name_scope("nce"):
            Wshape = (self.config.vocab_size, self.config.embed_size)
            bshape = (self.config.vocab_size)
            std = 1.0/(self.config.embed_size ** 0.5)
            Winit = tf.truncated_normal(shape=Wshape,
                                        stddev=std)
            binit = tf.zeros(bshape)
            self.W = tf.get_variable("nce_weight", dtype=tf.float32, initializer=Winit)
            self.b = tf.get_variable("nce_bias", dtype=tf.float32, initializer=binit)

            with tf.name_scope("loss"):
                embed = tf.nn.embedding_lookup(self.embed_matrix,
                                           self.center_words,
                                           name='embed')
                # define loss function to be NCE loss function
                self.loss = tf.reduce_mean(tf.nn.nce_loss(weights=self.W,
                                                          biases=self.b,
                                                          labels=self.target_words,
                                                          inputs=embed,
                                                          num_sampled=self.config.num_sampled,
                                                          num_classes=self.config.vocab_size), name='loss')

    def _create_optimizer(self):
        with tf.name_scope("train"):
            opt = tf.train.GradientDescentOptimizer(self.config.lr)
            self.optimizer = opt.minimize(self.loss,
                                          global_step=self.global_step)

    def _create_summaries(self):
        with tf.name_scope("summaries"):
            tf.summary.scalar("loss", self.loss)
            tf.summary.histogram("histogram_nce_weight",self.W)
            tf.summary.histogram("histogram_nce_bias",self.b)
            self.summary_op = tf.summary.merge_all()

    def build_graph(self):
        """ Build the graph for our model """
        self.graph = tf.Graph()
        with self.graph.as_default():
            self.global_step = tf.Variable(0,
                               dtype=tf.int32,
                               trainable=False,
                               name='global_step')
            self.saver = tf.train.Saver()
            save_dir = 'checkpoints/'
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            self.save_path = os.path.join(save_dir, 'best_validation')
            
            
            self._create_placeholders()
            self._create_embedding()
            self._create_loss()
            self._create_optimizer()
            self._create_summaries()


def train_model(model, batch_gen,log_path=None,visualization=False):

    initial_step = 0
    skip_step = model.config.skip_step
    if log_path is None:
        log_path = newlogname()
        print("\n&&&&&logdir={}&&&&&".format(log_path))
    with tf.Session(graph=model.graph) as sess:
        print("\n ==Beggining training==")
        summary_writer = tf.summary.FileWriter(log_path,sess.graph)
        tf.global_variables_initializer().run()
        ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/checkpoint'))
        if ckpt and ckpt.model_checkpoint_path:
            model.saver.restore(sess, ckpt.model_checkpoint_path)

        total_loss = 0
        initial_step = model.global_step.eval()
        for index in range(model.config.num_train_steps):
            centers, targets = batch_gen.__next__()
            feed_dict = {model.center_words: centers, model.target_words: targets}
            loss_batch, _, summary = sess.run([model.loss, model.optimizer, model.summary_op],
                                              feed_dict=feed_dict)
            summary_writer.add_summary(summary, global_step=index)
            summary_writer.flush()
            total_loss += loss_batch
            sys.stdout.write('\r{} / {} : loss = {}'.format(index,
                                                            model.config.num_train_steps,
                                                            total_loss / max(index, 1)))
            sys.stdout.flush()
            if (index + 1) % skip_step == 0:
                print('\nAverage loss at step {}: {:5.1f}'.format(index, total_loss / skip_step))
                total_loss = 0.0
                model.saver.save(sess=sess, save_path=model.save_path)
                
        if visualization:        
            final_embed_matrix = sess.run(model.embed_matrix)
            embedding_var = tf.Variable(final_embed_matrix[:1000], name='embedding')
            sess.run(embedding_var.initializer)
            config = projector.ProjectorConfig()
            new_summary_writer = tf.summary.FileWriter('processed')
            embedding = config.embeddings.add()
            embedding.tensor_name = embedding_var.name
            #link this tensor to its metadata file, in this case the first 500 words of vocab
            embedding.metadata_path = './processed/vocab_1000.tsv'
            # saves a configuration file that TensorBoard will read during startup.
            projector.visualize_embeddings(new_summary_writer, config)
            saver_embed = tf.train.Saver([embedding_var])
            saver_embed.save(sess, './processed/model3.ckpt', 1)

In [37]:
c = Config(num_train_steps)
batch_gen = process_data(c.vocab_size,c.batch_size,c.skip_window)
m = SkipGramModel(c)
train_model(m,batch_gen,visualization=True)

Dataset ready

&&&&&logdir=./graphs/05-03-2017_22-05-35&&&&&

 ==Beggining training==
199 / 200 : loss = 280.7931454912502

In [39]:
#!tensorboard --logdir=./graphs/05-03-2017_22-05-35 --port=8008

Starting TensorBoard b'41' on port 8008
(You can navigate to http://127.0.1.1:8008)
^CTraceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 11, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/tensorboard/tensorboard.py", line 151, in main
    tb_server.serve_forever()
  File "/usr/lib/python3.5/socketserver.py", line 232, in serve_forever
    ready = selector.select(poll_interval)
  File "/usr/lib/python3.5/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
KeyboardInterrupt



In [40]:
#!tensorboard --logdir=./processed --port=8008


Starting TensorBoard b'41' on port 8008
(You can navigate to http://127.0.1.1:8008)
^CTraceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 11, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/tensorboard/tensorboard.py", line 151, in main
    tb_server.serve_forever()
  File "/usr/lib/python3.5/socketserver.py", line 232, in serve_forever
    ready = selector.select(poll_interval)
  File "/usr/lib/python3.5/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
KeyboardInterrupt

